# Metody Obliczeniowe w Nauce i Technice
# Laboratorium 2: Rozwiązywanie układów równań liniowych
## Przemysław Roman

In [83]:
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy

def time_exec(func, *args):
    start_time = time.time()
    result = func(*args)
    return result, time.time() - start_time

## Zadanie 1 Metoda Gaussa-Jordana

Napisz i sprawdź funkcję rozwiązującą układ równań liniowych n × n metodą Gaussa-
Jordana. Dla rozmiarów macierzy współczynników większych niż 500 × 500 porównaj
czasy działania zaimplementowanej funkcji z czasami uzyskanymi dla wybranych funkcji
bibliotecznych.

Funkcje pomocnicze

In [84]:
# układa rzędy tak aby ilość wiodących zer w rzędach była niemalejąca idąc od góry do dołu układu równań
def order_rows_by_leading_zeros(AB, n):
  pre_zeros = [0] * n

  for i in range(n):
    for j in range(n):
      if AB[i,j] != 0:
        break
      pre_zeros[i] += 1

  sorted_indexes = [i for _, i in sorted(zip(pre_zeros, range(n)))]
  return np.array([AB[i] for i in sorted_indexes])

# sprawia, że wszystkie elementy w tablicy są w zakresie [0, 1]
def scaling(AB, n):
    for i in range(n):
        _max = max(AB[i])
        AB[i] /= np.max(AB[i])

def find_max_abs_scalar_i(AB, n, j):
    max_i = -1
    for i in range(j+1, n):
        if AB[i,j] > AB[max_i,j]:
            max_i = i
    return max_i

def partial_pivoting(AB, n, i):
    max_i = find_max_abs_scalar_i(AB, n, i)
    if max_i != -1:
        AB[[i, max_i]] = AB[[max_i, i]]

Implementacja algorytmu Gaussa-Jordana

In [85]:
# i - rząd, j - kolumna
def gauss_jordan(_AB, inplace=False):
    n = _AB.shape[0]
    if inplace:
        AB = _AB
    else:
        AB = deepcopy(_AB) # żeby nie niszczyć macierzy wejściowej
    AB = order_rows_by_leading_zeros(AB, n)
    scaling(AB, n)

    for i in range(n):
        partial_pivoting(AB, n, i)

        if AB[i,i] != 0:
            # redukcja pivotu do 1
            scalar = 1 / AB[i,i]
            AB[i] *= scalar

            # redukcja współczynników do 0, które znajdują się w tej samej kolumnie co pivot
            for k in range(n):
                if k != i:
                    scalar = AB[k,i] / AB[i,i]
                    AB[k] -= scalar * AB[i]

    return AB[:,-1] # ostatnia kolumna zawiera wyniki

Dane testowe

In [86]:
np.random.seed(17)
n = 500
A = np.random.random((n, n))
B = np.random.random((n, 1))
AB = np.concatenate((A, B), axis=1)

Testy poprawności rozwiązania oraz czasu wykonania

In [87]:
def get_absolute_errors_sum(vals, expected_vals):
    return np.sum(np.absolute(vals - expected_vals))

X1, t1 = time_exec(gauss_jordan, AB)
print(f'Moja implementacja: {t1}[s]')
X2, t2 = time_exec(np.linalg.solve, A, B)
X2 = X2.flatten()
print(f'np.linalg.solve: {t2}[s]')
print('Sumaryczny błąd względny:', get_absolute_errors_sum(X1, X2))

Moja implementacja: 1.0411553382873535[s]
np.linalg.solve: 0.002430438995361328[s]
Sumaryczny błąd względny: 3.808764805282583e-11


## Zadanie 2 Faktoryzacja LU

Napisz i przetestuj funkcję dokonującą faktoryzacji $ \textbf{A = LU} $ macierzy $ \textbf{A} $ (bez poszuki-
wania elementu wiodącego). Sprawdź poprawność wyniku obliczając $ \textbf{||A − LU||} $. Zadbaj
o to żeby implementacja była in-situ. Elementy macierzy $ \textbf{L} $ to współczynniki mnożenia
umożliwiające wyzerowanie odpowiedniego współczynnika macierzy $ \textbf{A} $ w trakcie procesu
eliminacji.

Implementacja algorytmu faktoryzacji LU macierzy

In [88]:
def factorize_matrix(A, inplace=False):
    n = A.shape[0]
    if inplace:
        LU = A
    else:
        LU = deepcopy(A) # żeby nie niszczyć macierzy wejściowej
    LU = order_rows_by_leading_zeros(LU, n)

    for i in range(n):
        if LU[i,i] != 0:
            for k in range(i+1, n):
                if k != i:
                    scalar = LU[k,i] / LU[i,i]
                    LU[k] -= scalar * LU[i]
                    LU[k,i] = scalar

    return LU

def split_LU(LU):
    n = LU.shape[0]
    L = deepcopy(LU)
    U = deepcopy(LU)
    for i in range(n):
        L[i,i] = 1
        for j in range(i+1, n):
            L[i,j] = 0
            U[j,i] = 0

    return L, U

Dane testowe

In [89]:
np.random.seed(17)
n = 500
A = np.random.random((n, n))
LU, t = time_exec(factorize_matrix, A)
L, U = split_LU(LU)

Testy poprawności rozwiązania oraz czasu wykonania

In [90]:
def get_matrix_factorization_error(A, L, U):
    return np.linalg.det(A - np.matmul(L, U))

print(f'Czas wykonania: {t}[s]')
print('Błąd faktoryzacji macierzy:', get_matrix_factorization_error(A, L, U))

Czas wykonania: 0.5318150520324707[s]
Błąd faktoryzacji macierzy: 0.0
